In [2]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt

# Import the random module.
import random

# Import the NumPy module.
import numpy as np

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

#Import requests
import requests

In [3]:
# Create a new set of 2000 random latitude and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=20)
lngs = np.random.uniform(low=-180.000, high=180.000, size=20)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
#get the nearest city using the citipy module

for coordinate in coordinates:
    print(citipy.nearest_city(coordinate[0], coordinate[1]).city_name,
          citipy.nearest_city(coordinate[0], coordinate[1]).country_code)

kahului us
khatanga ru
dikson ru
rikitea pf
morondava mg
katsuura jp
misratah ly
khandyga ru
atuona pf
palabuhanratu id
naze jp
luganville vu
nikolskoye ru
mar del plata ar
albany au
sao joao da barra br
brae gb
albany au
busselton au
klaksvik fo


In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

19

In [7]:
# Import the API key.
from config import weather_api_key2
import requests

In [8]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key2

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
HTTP_STATUS_OK = 200
HTTP_STATUS_NOT_FOUND = 404

# Loop through all the cities in the list.
for i, city in enumerate(cities[:3]):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
# Run an API request for each of the cities.
    try:
        response = requests.get(city_url)
        status_code = response.status_code
        if status_code == HTTP_STATUS_OK:
            city_weather = response.json()
        # Parse out the needed data.
            city_lat = city_weather["coord"]["lat"]
            city_lng = city_weather["coord"]["lon"]
            city_max_temp = city_weather["main"]["temp_max"]
            city_humidity = city_weather["main"]["humidity"]
            city_clouds = city_weather["clouds"]["all"]
            city_wind = city_weather["wind"]["speed"]
            city_country = city_weather["sys"]["country"]
            city_description = city_weather["weather"][0]["description"]
            # Convert the date to ISO standard.
            city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            # Append the city information into city_data list.
            city_data.append({"City": city.title(),
                              "Lat": city_lat,
                              "Lng": city_lng,
                              "Max Temp": city_max_temp,
                              "Humidity": city_humidity,
                              "Cloudiness": city_clouds,
                              "Wind Speed": city_wind,
                              "Current Description": city_description,
                              "Country": city_country, "Date": city_date})
        elif status_code == HTTP_STATUS_NOT_FOUND:
            print("City not found. Skipping...")
            print(status_code)
        else:
            raise ValueError (f"Unexpected error HTTP status: {status_code}")
# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        print(status_code)
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")    

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | kahului
Processing Record 2 of Set 1 | khatanga
Processing Record 3 of Set 1 | dikson
-----------------------------
Data Retrieval Complete      
-----------------------------


In [10]:
#Add weather data to dataframe
city_data_df = pd.DataFrame(city_data)

In [11]:
#Create new column order
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]

In [12]:
#apply new column order
city_data_df = city_data_df[new_column_order]

city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kahului,US,20.8947,-156.4700,78.82,68,20,13.80,few clouds
1,Khatanga,RU,71.9667,102.5000,-35.86,99,21,3.15,few clouds
2,Dikson,RU,73.5069,80.5464,3.94,93,100,18.72,light snow


In [14]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Databasexxx.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")
